In [ ]:
import nltk as nk ## natural language processing
import numpy as np
from bs4 import BeautifulSoup ## parsing html

In [ ]:
## iterate over all files in MusicVocab and extract sentences
## store sentences in a plain text file
## begin co-occurence counting